# Case Study - Freddie Mac Bonds


## 1. Pricing the Callable Bond


### Data

Use the data from the following files.
* `../data/callable_bonds_2025-02-13.xlsx`
* `../data/discount_curve_2025-02-13.xlsx`


The data contains info on the following bonds.

`Callable`
* `FHLMC 4.41 01/28/30` is a callable bond, and it is the primary object of our analysis.


In [192]:
FILE_BOND = 'callable_bonds_2025-02-13.xlsx'
FILE_CURVE = 'discount_curve_2025-02-13.xlsx'

KEY_CALLABLE = 'FHLMC 4.41 01/28/30'

### Bond Info


In [193]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from math import log, sqrt, erf
info = pd.read_excel(FILE_BOND,sheet_name='info').set_index('info')
info_core = info[[KEY_CALLABLE]]
info_core.style.format('{:.2%}',subset=pd.IndexSlice[["Cpn Rate"], :]).format('{:,.0f}',subset=pd.IndexSlice[["Amount Issued"], :]).format('{:%Y-%m-%d}',subset=pd.IndexSlice[["Date Quoted","Date Issued","Date Matures","Date Next Call","Date of First Possible Call"], :])

,FHLMC 4.41 01/28/30
info,
CUSIP,3134HA4V2
Issuer,FREDDIE MAC
Maturity Type,CALLABLE
Issuer Industry,GOVT AGENCY
Amount Issued,"10,000,000"
Cpn Rate,4.41%
Cpn Freq,2
Date Quoted,2025-02-13
Date Issued,2025-01-28


### Quoted Values


In [194]:
quotes = pd.read_excel(FILE_BOND,sheet_name='quotes').set_index('quotes')
quotes_core = quotes[[KEY_CALLABLE]]
quotes_core.style.format('{:.2f}', subset=pd.IndexSlice[quotes.index[1:], :]).format('{:%Y-%m-%d}', subset=pd.IndexSlice['Date Quoted', :])

,FHLMC 4.41 01/28/30
quotes,
Date Quoted,2025-02-13
TTM,4.96
Clean Price,99.89
Dirty Price,100.09
Accrued Interest,0.20
YTM Call,4.45
YTM Maturity,4.43
Duration,4.50
Modified Duration,4.40


### Discount Curves


In [195]:
discs = pd.read_excel(FILE_CURVE,sheet_name='discount curve').set_index('ttm')
display(discs.head())
display(discs.tail())

,maturity date,spot rate,discount
ttm,,,
0.5,2025-08-13,0.043743,0.978597
1.0,2026-02-13,0.042890,0.958451
1.5,2026-08-13,0.042238,0.939228
2.0,2027-02-13,0.041843,0.920515
2.5,2027-08-13,0.041632,0.902117


,maturity date,spot rate,discount
ttm,,,
28.0,2053-02-13,0.040185,0.328231
28.5,2053-08-13,0.040051,0.322978
29.0,2054-02-13,0.039916,0.317851
29.5,2054-08-13,0.039791,0.312766
30.0,2055-02-13,0.039665,0.307802


### 1.1.

Use the discount curve data to price both the `callable` and `reference` bonds.

Also calculate the price of the `hypothetical` bonds, where we consider a non-callable version of the callable bond with 
* maturity unchanged
* maturity at the call date.


In [196]:
def yearfrac_act365(d1, d2):
    return (pd.Timestamp(d2) - pd.Timestamp(d1)).days / 365.0

def build_cpn_schedule(settle_date, maturity_date, cpn_freq):
    "Build cpn payment dates after settle_date and up to maturity_date, given cpn_freq (number of payments per year)"
    settle_date = pd.Timestamp(settle_date)
    maturity_date = pd.Timestamp(maturity_date)
    
    step_months = int(round(12/cpn_freq))
    dates = []
    d = maturity_date
    while d > settle_date:
        dates.append(d)
        d = d - relativedelta(months=step_months)

    dates = sorted(dates)
    return dates

def interp_df(ttm_grid, df_grid, ttm_targets):
    "Linear interpolation of discount factors in ttm space"
    ttm_grid = np.asarray(ttm_grid, dtype=float)
    df_grid = np.asarray(df_grid, dtype=float)
    ttm_targets = np.asarray(ttm_targets, dtype=float)

    if ttm_targets.min() < ttm_grid.min() or ttm_targets.max() > ttm_grid.max():
        raise ValueError("Target TTM out of bounds of grid")
    return np.interp(ttm_targets, ttm_grid, df_grid)

def pv_bond_from_curve(settle_date, cpn_rate, freq, maturity_date, curve_df, face=100.0):
    "PV (dirty price per 100) of a noncallable fixed cpn bond using discount factors"
    pay_dates = build_cpn_schedule(settle_date, maturity_date, freq)
    t = np.array([yearfrac_act365(settle_date, d) for d in pay_dates])
    dfs = interp_df(curve_df.index.values, curve_df['discount'].values, t)

    cpn = face * cpn_rate / freq
    cfs = np.full_like(t, cpn, dtype=float)
    cfs[-1] += face

    dirty = float(np.sum(cfs * dfs))
    return dirty, pay_dates, t, dfs, cfs

def price_callable_deterministic(settle_date, cpn_rate, freq, maturity_date, call_date, call_price, curve_df, face=100.0):
    "Price of a callable bond assuming deterministic exercise at call date"
    settle_date = pd.Timestamp(settle_date)
    call_date = pd.Timestamp(call_date)
    maturity_date = pd.Timestamp(maturity_date)

    dirty_to_mat, *_ = pv_bond_from_curve(settle_date, cpn_rate, freq, maturity_date, curve_df, face)
    dirty_to_call, pay_dates_call, t_call_sched, dfs_call_sched, cfs_call_sched = pv_bond_from_curve(settle_date, cpn_rate, freq, call_date, curve_df, face)

    rem_pay_dates = build_cpn_schedule(call_date, maturity_date, freq)
    rem_t = np.array([yearfrac_act365(settle_date, d) for d in rem_pay_dates], dtype=float)

    #DF(0, t_call)
    t_call = yearfrac_act365(settle_date, call_date)
    df_call = interp_df(curve_df.index.values, curve_df['discount'].values, [t_call])[0]

    #Discount Factors to remaining dates from today
    rem_dfs_today = interp_df(curve_df.index.values, curve_df['discount'].values, rem_t)

    #Convert to discount factors from call date
    rem_dfs_from_call = rem_dfs_today / df_call

    cpn = face * cpn_rate / freq
    rem_cfs = np.full_like(rem_t, cpn, dtype=float)
    rem_cfs[-1] += face

    #Value at call date of holding to maturiy
    val_hold_at_call = float(np.sum(rem_cfs * rem_dfs_from_call))
    called = val_hold_at_call > call_price
    dirty_callable = dirty_to_call if called else dirty_to_mat
    return dirty_callable, called, dirty_to_mat, dirty_to_call, val_hold_at_call

In [197]:
curve = discs.copy()
curve.loc[0.0, 'discount'] = 1.0
curve = curve.sort_index()

settle = quotes_core.loc['Date Quoted', KEY_CALLABLE]
cpn_rate = float(info_core.loc['Cpn Rate', KEY_CALLABLE])
freq = int(info_core.loc['Cpn Freq', KEY_CALLABLE])
maturity = info_core.loc['Date Matures', KEY_CALLABLE]
call_date = info_core.loc['Date Next Call', KEY_CALLABLE]
call_price = float(info_core.loc['Strike', KEY_CALLABLE])

accrued = float(quotes_core.loc['Accrued Interest', KEY_CALLABLE])
market_clean = float(quotes_core.loc['Clean Price', KEY_CALLABLE])
market_dirty = float(quotes_core.loc['Dirty Price', KEY_CALLABLE])

#### Prices: ####

#Hypothetical noncallable price (dirty)
dirty_nc_mat, *_ = pv_bond_from_curve(settle, cpn_rate, freq, maturity, curve)
dirty_nc_call, *_ = pv_bond_from_curve(settle, cpn_rate, freq, call_date, curve)
dirty_callable, called_flag, dirty_to_mat, dirty_to_call, hold_val_at_call = price_callable_deterministic(settle, cpn_rate, freq, maturity, call_date, call_price, curve)

#Convert to clean using accrued interest
clean_nc_mat = dirty_nc_mat - accrued
clean_nc_call = dirty_nc_call - accrued
clean_callable = dirty_callable - accrued

out = pd.DataFrame({
    "Instrument": ['Market (from sheet)',
    'Hypothetical Noncallable (to maturity)', 'Hypothetical Noncallable (to call)', 'Callable (deterministic)'],
    "Clean Price": [market_clean, clean_nc_mat, clean_nc_call, clean_callable],
    "Dirty Price": [market_dirty, dirty_nc_mat, dirty_nc_call, dirty_callable],
})

extra = pd.DataFrame({
    "Callable Diagnostics": [
        f'Call Decision Date: {call_date.date()}',
        f'Value of Holding to Maturity at Call Date: ${hold_val_at_call:.2f}',
        f'Call Price (strike): ${call_price:.2f} per 100.0',
        f'Issuer Calls Under Curve? {"Yes" if called_flag else "No"}'
    ]
})

display(out.style.format({'Clean Price': '${:,.2f}', 'Dirty Price': '${:,.2f}'}))
display(extra)

,Instrument,Clean Price,Dirty Price
0,Market (from sheet),$99.89,$100.09
1,Hypothetical Noncallable (to maturity),$101.21,$101.40
2,Hypothetical Noncallable (to call),$100.70,$100.90
3,Callable (deterministic),$100.70,$100.90


,Callable Diagnostics
0,Call Decision Date: 2028-01-28
1,Value of Holding to Maturity at Call Date: $10...
2,Call Price (strike): $100.00 per 100.0
3,Issuer Calls Under Curve? Yes


### 1.2.

Calculate the forward price of the `hypothetical` bond as of the date that the `callable` bond can be exercised.

Use the information from the discount curve (and associated forward curve) to calculate this forward price.


In [198]:
def forward_dirty_price_from_curve(settle_date, cpn_rate, freq, maturity_date, fwd_date, curve_df, face=100.0):
    "Forward dirty price for delivery at fwd_date of a noncallable fixed cpn bond maturing at maturity_date using discount factors,"
    "subtracts cashflows with paydate <= fwd_date"
    settle_date = pd.Timestamp(settle_date)
    fwd_date = pd.Timestamp(fwd_date)

    #Spot PV of full bond (dirty)
    dirty_spot, pay_dates, t, dfs, cfs = pv_bond_from_curve(settle_date, cpn_rate, freq, maturity_date, curve_df, face)

    #PV of cashflows paid on/before delivery date
    paid_mask = np.array([pd.Timestamp(d) <= pd.Timestamp(fwd_date) for d in pay_dates])
    pv_paid = float(np.sum(cfs[paid_mask] * dfs[paid_mask]))

    #DF(0, T)
    T = yearfrac_act365(settle_date, fwd_date)
    df_T = float(interp_df(curve_df.index.values, curve_df['discount'].values, [T])[0])

    fwd_dirty = (dirty_spot - pv_paid) / df_T
    return fwd_dirty, pv_paid, df_T, T

fwd_dirty, pv_coupons_to_T, df_T, T = forward_dirty_price_from_curve(
    settle, cpn_rate, freq, maturity, call_date, curve)

print(f'Forward date T {pd.Timestamp(call_date).date()} is {T:.4f} years')
print(f'PV of coupons paid on/before T: ${pv_coupons_to_T:.2f}')
print(f'DF(0, T): {df_T:.6f}')
print(f'Forward dirty price at call date (ex-coupon): ${fwd_dirty:.2f}')   

Forward date T 2028-01-28 is 2.9562 years
PV of coupons paid on/before T: $12.33
DF(0, T): 0.885665
Forward dirty price at call date (ex-coupon): $100.57


Under the discount curve, the hypothetical noncallable-to-2030 bond is expected to be worth $100.57 on 2028-01-28. Since the call strike is 100, and the forward value at the call date is > 100, it is optimal for the issuer to call in this deterministic/no-vol setting.

### 1.3.

The provided implied vol corresponds to the implied vol of the **rate**. Specifically,
* the forward rate corresponding to the time of expiration.
* continuously compounded.

Use the duration approximation to get the approximate implied vol corresponding to the forward price.

$$\sigma_{\text{bond fwd price}} \approx D \times \sigma_{\text{fwd rate}}\times f(T_1)$$

where $f(T_1)$ is the continuously-compounded (instantaneous) forward rate at time $T_1$.
* If you're struggling with the forward rate calc, just usse the provided spot rate at $T_1$; it will be a close approximation in this example.
* In this approximation, use the quoted duration from the table. (Yes, this is a bit circular, but we don't want to get bogged down with a duration calculation at this point.)

Report the implied vol of the bond's forward price.


In [199]:
D_used = float(quotes_core.loc['Modified Duration', KEY_CALLABLE])
sigma_fwd_rate = float(quotes_core.loc['Implied Vol', KEY_CALLABLE]) / 100.0

#Time to call (T1)
T1 = yearfrac_act365(settle, call_date)

#Approx forward rate at T1 using spot rate from curve
DF_T1 = float(np.interp(T1, curve.index.values, curve['discount'].values))
f_T1 = -np.log(DF_T1) / T1

#Duration-based Approx
sigma_bond_fwd_price = D_used * sigma_fwd_rate * f_T1

print(f'Modified Duration (D): {D_used:.3f}')
print(f'Forward Rate Volatility: {sigma_fwd_rate*100:.2f}%')
print(f'Approx Forward Rate at T1: {f_T1*100:.2f}%')
print(f'Implied Volatility of Bond Forward Price: {sigma_bond_fwd_price*100:.2f}%')

Modified Duration (D): 4.399
Forward Rate Volatility: 23.88%
Approx Forward Rate at T1: 4.11%
Implied Volatility of Bond Forward Price: 4.31%


Using the duration approximation, the 23.9% implied vol of the forward rate translates into an implied vol of only ~4.3% for the bond's forward price, illustrating that bond price risk at the call date is substantitally dampened by duration and the level of interest rates.

### 1.4.

For the `callable` bond, report Black's value of the embedded call option.
* Use this to report the value of the `callable` bond.
* How does it compare to the actual market price?

For the calculation of the option, use...
* the quoted `Implied Vol` calculated above.
* forward price of the `hypothetical` bond calculated above.
* provided discount factor

#### Simplifications
Note that in this calculation we are making a few simplifications.
* We are simplifying that the `callable` bond is European exercise with an exercise date as reported in `Date Next Call` above. 
* In reality, it is Bermudan, with quarterly exercise dates after the first exercise date.
* The time-to-exercise is not a round number, but you only have discount factors at rounded time-to-maturities. Just use the closest discount factor.


In [200]:
def norm_cdf(x):
    return 0.5 * (1.0 +erf(x/sqrt(2.0)))

def black_call_on_fwd(F, K, sigma, T, DF):
    "Black 76 call option on a forward"
    if T<=0:
       return DF * max(F-K, 0.0)
    if sigma <=0:
        return DF * max(F-K, 0.0)
    vol_sqrtT = sigma * sqrt(T)
    d1 = ((log(F/K) + 0.5 * sigma**2 * T)) / vol_sqrtT
    d2 = d1 - vol_sqrtT
    call_price = DF * (F * norm_cdf(d1) - K * norm_cdf(d2))
    return call_price

T = yearfrac_act365(settle, call_date)
DF_T = float(np.interp(T, curve.index.values, curve['discount'].values))

F = fwd_dirty
K = call_price
sigma = sigma_bond_fwd_price

#Black Value of Embedded Call Option
call_option_pv = black_call_on_fwd(F, K, sigma, T, DF_T)

#Callable bond value using option-adjusted approach
P_nc_dirty = dirty_nc_mat
P_callable_dirty_black = P_nc_dirty - call_option_pv

P_call_clean_black = P_callable_dirty_black - accrued

#Compare to Market
print(f'T (years to call): {T:.4f}')
print(f'DF(0, T): {DF_T:.4f}')
print(f'Forward Price at T (ex-coupon): ${F:.2f}')
print(f'Strike (call price): ${K:.2f}')
print(f'Implied Vol of Bond Forward Price: {sigma*100:.2f}%\n')
print(f'Black 76 Call Option Price (PV today): ${call_option_pv:.2f}')
print(f'Callable Bond Price (Option-Adjusted, Black 76): ${P_call_clean_black:.2f}')

print(f'Market Dirty Price: ${market_dirty:.2f}')
print(f'Market Clean Price: ${market_clean:.2f} \n')

#Model - Market
print(f'Dirty Difference: ${P_callable_dirty_black - market_dirty:.2f}')
print(f'Clean Difference: ${P_call_clean_black - market_clean:.2f}')

T (years to call): 2.9562
DF(0, T): 0.8857
Forward Price at T (ex-coupon): $100.57
Strike (call price): $100.00
Implied Vol of Bond Forward Price: 4.31%

Black 76 Call Option Price (PV today): $2.89
Callable Bond Price (Option-Adjusted, Black 76): $98.32
Market Dirty Price: $100.09
Market Clean Price: $99.89 

Dirty Difference: $-1.57
Clean Difference: $-1.57


### 1.5.

Calculate the YTM of the callable bond, assuming that...
* it can never be called. (This is the `hypothetical` bond we analyzed above.)
* it will certainly be called.

How do these compare to the quoted YTM Called and YTM Maturity in the table?


In [201]:
#Build Cashflows
def bond_cashflows(settle_date, cpn_rate, freq, maturity_date, face=100.0):
    "Build cashflow schedule and amounts for a fixed cpn bond"
    settle_date = pd.Timestamp(settle_date)
    end_date = pd.Timestamp(maturity_date)

    step_months = int(round(12/freq))
    dates = []
    d = end_date
    while d > settle_date:
        dates.append(d)
        d = d - relativedelta(months=step_months)
    dates = sorted(dates)
    t = np.array([(pd.Timestamp(d) - settle_date).days / 365.0 for d in dates], dtype=float)

    cpn = face * cpn_rate / freq
    cfs = np.full_like(t, cpn, dtype=float)
    cfs[-1] += face
    return t, cfs, dates

def price_from_ytm(y, t, cfs, freq):
    return float(np.sum(cfs / (1.0 + y/freq)**(t*freq)))

def ytm_from_price(target_price, t, cfs, freq, y_low=-0.99, y_high=1.00, max_iter=200):
    f_low = price_from_ytm(y_low, t, cfs, freq) - target_price
    f_high = price_from_ytm(y_high, t, cfs, freq) - target_price
    if f_low * f_high > 0:
        raise ValueError("Target price out of bounds for YTM search")
    
    for _ in range(max_iter):
        y_mid = (y_low + y_high) / 2.0
        f_mid = price_from_ytm(y_mid, t, cfs, freq) - target_price
        if abs(f_mid) < 1e-8:
            return y_mid
        if f_low * f_mid < 0:
            y_high = y_mid
            f_high = f_mid
        else:
            y_low = y_mid
            f_low = f_mid
    return y_mid

settle = quotes_core.loc['Date Quoted', KEY_CALLABLE]
cpn_rate = float(info_core.loc['Cpn Rate', KEY_CALLABLE])
freq = int(info_core.loc['Cpn Freq', KEY_CALLABLE])
maturity = info_core.loc['Date Matures', KEY_CALLABLE]
call_date = info_core.loc['Date Next Call', KEY_CALLABLE]

P_mkt_dirty = float(quotes_core.loc['Dirty Price', KEY_CALLABLE])
ytm_call_quoted = float(quotes_core.loc['YTM Call', KEY_CALLABLE]) / 100.0
ytm_mat_quoted = float(quotes_core.loc['YTM Maturity', KEY_CALLABLE]) / 100.0

t_mat, cfs_mat, _ = bond_cashflows(settle, cpn_rate, freq, maturity)
ytm_to_maturity = ytm_from_price(P_mkt_dirty, t_mat, cfs_mat, freq)

t_call, cfs_call, _ = bond_cashflows(settle, cpn_rate, freq, call_date)
ytm_to_call = ytm_from_price(P_mkt_dirty, t_call, cfs_call, freq)

print(f'Market Dirty Price: ${P_mkt_dirty:.2f}')
print(f'Quoted YTM to Call: {ytm_call_quoted*100:.2f}%')
print(f'Quoted YTM to Maturity: {ytm_mat_quoted*100:.2f}%')
print(f'Calculated YTM to Call: {ytm_to_call*100:.2f}%')
print(f'Calculated YTM to Maturity: {ytm_to_maturity*100:.2f}%\n')

print("Differences (Computed - Quoted):")
print(f"To Call: {(ytm_to_call - ytm_call_quoted)*100:.4f}%")
print(f"To Maturity: {(ytm_to_maturity - ytm_mat_quoted)*100:.4f}%")

Market Dirty Price: $100.09
Quoted YTM to Call: 4.45%
Quoted YTM to Maturity: 4.43%
Calculated YTM to Call: 4.45%
Calculated YTM to Maturity: 4.43%

Differences (Computed - Quoted):
To Call: -0.0006%
To Maturity: -0.0026%


Under the assumption that the bond is never called, the YTM to maturity matches the quoted YTM Maturity, while under certain exercises the YTM to call matches the quoted YTM call, confirming that the quoted yields correspond to these two limiting cases of call behavior.

### 1.6.

Calculate the duration of...
* the `hypothetical` bond
* the `callable` bond

How do these compare to the quoted duration in the table?

For the callable bond, calculate duration numerically by modifying the spot rates up and down by 1bp and seeing how it changes the valuation of parts `1.1`-`1.3`.


In [202]:
BUMP_BP = 1.0
dy = BUMP_BP / 10000.0 #1 basis point in decimal form

def bump_curve(curve_df, bump):
    "Bump discount curve by bump_bp basis points in spot rate space"
    out = curve_df.copy()
    if 0.0 not in out.index:
        out.loc[0.0, 'spot rate'] = 0.0
        out.loc[0.0, 'discount'] = 1.0
    out = out.sort_index()

    t = out.index.values.astype(float)
    r = out['spot rate'].values.astype(float)
    out['discount'] = np.where(t ==0.0, 1.0, np.exp(- (r + bump) * t))

    out.loc[t!=0.0, 'spot rate'] = r[t!=0.0] + bump
    return out
                               
def effective_duration(P_up, P_0, P_down, dy):
    "Effective duration approximation using bumped prices"
    return (P_down - P_up) / (2 * P_0 * dy)

#Base Curve + Bumped Curves
curve0 = curve.copy()
curve_up = bump_curve(curve0, dy)
curve_down = bump_curve(curve0, -dy)

#Hypothetical noncallable price (dirty) under each curve
P_nc_0, *_ = pv_bond_from_curve(settle, cpn_rate, freq, maturity, curve0)
P_nc_up, *_ = pv_bond_from_curve(settle, cpn_rate, freq, maturity, curve_up)
P_nc_down, *_ = pv_bond_from_curve(settle, cpn_rate, freq, maturity, curve_down)

Dur_nc = effective_duration(P_nc_up, P_nc_0, P_nc_down, dy)

#Callable Bond
P_call_0, called0, *_ = price_callable_deterministic(settle, cpn_rate, freq, maturity, call_date, call_price, curve0)
P_call_up, called0, *_ = price_callable_deterministic(settle, cpn_rate, freq, maturity, call_date, call_price, curve_up)
P_call_down, called0, *_ = price_callable_deterministic(settle, cpn_rate, freq, maturity, call_date, call_price, curve_down)

Dur_call = effective_duration(P_call_up, P_call_0, P_call_down, dy)

#Forward Price Changes Under Bump
F0, pv_paid0, DF_T0, T0 = forward_dirty_price_from_curve(settle, cpn_rate, freq, maturity, call_date, curve0)
F_up, pv_paid_up, DF_T_up, T_up = forward_dirty_price_from_curve(settle, cpn_rate, freq, maturity, call_date, curve_up)
F_down, pv_paid_down, DF_T_down, T_down = forward_dirty_price_from_curve(settle, cpn_rate, freq, maturity, call_date, curve_down)

D_quoted = float(quotes_core.loc['Duration', KEY_CALLABLE])
sigma_rate = float(quotes_core.loc['Implied Vol', KEY_CALLABLE]) / 100.0

f0 = np.interp(T0, curve0.index.values, curve0['spot rate'].values)
fup = np.interp(T_up, curve_up.index.values, curve_up['spot rate'].values)
fdown = np.interp(T_down, curve_down.index.values, curve_down['spot rate'].values)

sigmaF0 = D_quoted * sigma_rate * f0
sigmaF_up = D_quoted * sigma_rate * fup
sigmaF_down = D_quoted * sigma_rate * fdown

Dur_quoted = float(quotes_core.loc['Duration', KEY_CALLABLE])
ModDur_quoted = float(quotes_core.loc['Modified Duration', KEY_CALLABLE])

print(f"Effective Duration (Noncallable): {Dur_nc:.4f}")
print(f"Effective Duration (Callable): {Dur_call:.4f}")
print(f"Quoted Duration: {D_quoted:.4f}")
print(f"Quoted Modified Duration: {ModDur_quoted:.4f}\n")
print(f'Callable Price at Base Curve: ${P_call_0:.2f}')
print(f'Callable Price at Up Bump: ${P_call_up:.2f}')
print(f'Callable Price at Down Bump: ${P_call_down:.2f}\n')
print(f"Forward Price at T0: ${F0:.2f}")
print(f"Forward Price at T_up: ${F_up:.2f}")
print(f"Forward Price at T_down: ${F_down:.2f}\n")
print(f"Implied Vol of Forward Price at T0: {sigmaF0*100:.2f}%")
print(f"Implied Vol of Forward Price at T_up: {sigmaF_up*100:.2f}%")
print(f"Implied Vol of Forward Price at T_down: {sigmaF_down*100:.2f}%")

Effective Duration (Noncallable): 4.4935
Effective Duration (Callable): 2.7952
Quoted Duration: 4.4967
Quoted Modified Duration: 4.3992

Callable Price at Base Curve: $100.90
Callable Price at Up Bump: $100.75
Callable Price at Down Bump: $100.81

Forward Price at T0: $100.57
Forward Price at T_up: $100.47
Forward Price at T_down: $100.50

Implied Vol of Forward Price at T0: 4.46%
Implied Vol of Forward Price at T_up: 4.47%
Implied Vol of Forward Price at T_down: 4.45%


**Observations:**
- The effective duration of the hypothethical noncallable bond is ~4.5, matching the quoted duration and validating the duration calculation
- The callable bond's effective duration is much lower than the noncallable bond's duration, which is due to the call option truncating cashflows when rates fall, limiting upside price sensitivity
- The quoted duration does not fully reflect the path-dependent behavior, whereas the numerically computed effective duration captures the callable bond's reduced interest rate sensitivity due to the call option

### 1.7.

Calculate the OAS of the `callable` bond.

How does it compare to the quoted OAS?

Recall that the OAS is the parallel shift in the spot curve needed to align the modeled value to the market quote.


In [203]:
P_mkt = market_dirty
quoted_oas = float(quotes_core.loc['OAS Spread', KEY_CALLABLE])

def shift_curve_on_discount(curve_df, spread_cc):
    out = curve_df.copy()
    if 0.0 not in out.index:
        out.loc[0.0, 'discount'] = 1.0
    out = out.sort_index()
    t = out.index.values.astype(float)
    df = out['discount'].values.astype(float)
    out['discount'] = np.where(t==0.0, 1.0, df * np.exp(-spread_cc * t))
    return out

def callable_price_black_shifted(spread):
    c_shift = shift_curve_on_discount(curve0, spread)
    P_nc_dirty, *_ = pv_bond_from_curve(settle, cpn_rate, freq, maturity, c_shift)
    F_shift, _, DF_T_shift, T_shift = forward_dirty_price_from_curve(
        settle, cpn_rate, freq, maturity, call_date, c_shift)
    call_pv = black_call_on_fwd(
        F_shift, call_price, sigma_bond_fwd_price, T_shift, DF_T_shift)
    return P_nc_dirty - call_pv

def solve_oas_bisect(low_bp=-500, high_bp=500, max_iter=200):
    low = low_bp / 10000.0
    high = high_bp / 10000.0

    f_low = callable_price_black_shifted(low) - P_mkt
    f_high = callable_price_black_shifted(high) - P_mkt
    if f_low * f_high > 0:
        raise ValueError("Market price out of bounds for OAS search")
    for _ in range(max_iter):
        mid = (low + high) / 2.0
        f_mid = callable_price_black_shifted(mid) - P_mkt
        if abs(f_mid) < 1e-8:
            return mid
        if f_low * f_mid < 0:
            high = mid
            f_high = f_mid
        else:
            low = mid
            f_low = f_mid
    return mid

oas_decimal = solve_oas_bisect()
oas_bp = oas_decimal * 10000.0

#Model Price Check
P_fit = callable_price_black_shifted(oas_decimal)
print(f'Modeled OAS (parallel shift): {oas_bp:.2f} bps')
print(f'Quoted OAS (from table): {quoted_oas:.2f} bps')
print(f'Difference (Modeled - Quoted): {oas_bp - quoted_oas:.2f} bps')
    

Modeled OAS (parallel shift): -44.57 bps
Quoted OAS (from table): -26.77 bps
Difference (Modeled - Quoted): -17.80 bps


### 1.8. Optional OTM Callables


There are a few other Freddie Mac callables that may be of interest.
* `FHLMC 0.97 01/28/28`
* `FHLMC 1.25 01/29/30`

Though these are technically callable, they are far out of the money (OTM). 
* Expiring in 3 months, though code below changes it to 6 monhts, to match coupon.
* These don't have interesting convexity due to being so far OTM.


In [204]:
# KEY_CALLABLE = 'FHLMC 1 1/4 01/29/30'
# KEY_CALLABLE = 'FHLMC 0.97 01/28/28'

In [205]:
info.style.format('{:.2%}',subset=pd.IndexSlice[["Cpn Rate"], :]).format('{:,.0f}',subset=pd.IndexSlice[["Amount Issued"], :]).format('{:%Y-%m-%d}',subset=pd.IndexSlice[["Date Quoted","Date Issued","Date Matures","Date Next Call","Date of First Possible Call"], :])

,FHLMC 0.97 01/28/28,FHLMC 1 1/4 01/29/30,FHLMC 4.41 01/28/30
info,,,
CUSIP,3134GW5F9,3134GWGK6,3134HA4V2
Issuer,FREDDIE MAC,FREDDIE MAC,FREDDIE MAC
Maturity Type,CALLABLE,CALLABLE,CALLABLE
Issuer Industry,GOVT AGENCY,GOVT AGENCY,GOVT AGENCY
Amount Issued,"30,000,000","25,000,000","10,000,000"
Cpn Rate,0.97%,1.25%,4.41%
Cpn Freq,2,2,2
Date Quoted,2025-02-13,2025-02-13,2025-02-13
Date Issued,2020-10-28,2020-07-29,2025-01-28


In [206]:
quotes.style.format('{:.2f}', subset=pd.IndexSlice[quotes.index[1:], :]).format('{:%Y-%m-%d}', subset=pd.IndexSlice['Date Quoted', :])

,FHLMC 0.97 01/28/28,FHLMC 1 1/4 01/29/30,FHLMC 4.41 01/28/30
quotes,,,
Date Quoted,2025-02-13,2025-02-13,2025-02-13
TTM,2.95,4.96,4.96
Clean Price,90.14,85.11,99.89
Dirty Price,90.19,85.16,100.09
Accrued Interest,0.04,0.05,0.20
YTM Call,54.24,85.40,4.45
YTM Maturity,4.57,4.65,4.43
Duration,2.92,4.81,4.50
Modified Duration,2.85,4.70,4.40


In [208]:
KEYS_OTM = ['FHLMC 0.97 01/28/28', 'FHLMC 1 1/4 01/29/30']
def next_coupon_date(settle_date, maturity_date, cpn_freq):
    "Calculate next coupon date after settle_date given maturity_date and cpn_freq"
    sched = build_cpn_schedule(settle_date, maturity_date, cpn_freq)
    return pd.Timestamp(sched[0]) if len(sched) > 0 else None

rows = []
for key in KEYS_OTM:
    settle_i = quotes.loc['Date Quoted', key]
    cpn_i = info.loc['Cpn Rate', key]
    freq_i = info.loc['Cpn Freq', key]
    maturity_i = info.loc['Date Matures', key]
    K = info.loc['Strike', key]

    exp_date = next_coupon_date(settle_i, maturity_i, freq_i)
    T = yearfrac_act365(settle_i, exp_date)

    P_nc_dirty, *_ = pv_bond_from_curve(settle_i, cpn_i, freq_i, maturity_i, curve)
    F, pv_paid, DF_T, _Tcheck = forward_dirty_price_from_curve(settle_i, cpn_i, freq_i, maturity_i, exp_date, curve)

    Dmod = float(quotes.loc['Modified Duration', key])
    sigma_rate = float(quotes.loc['Implied Vol', key]) / 100.0
    
    DF_exp = float(np.interp(T, curve.index.values, curve['discount'].values))
    f_T = -np.log(DF_exp) / T
    sigma_F = Dmod * sigma_rate * f_T

    opt_pv = black_call_on_fwd(F, K, sigma_F, T, DF_T)
    P_call_model = P_nc_dirty - opt_pv

    P_mkt_dirty = float(quotes.loc['Dirty Price', key])

    rows.append({
        'Bond': key,
        'Expiry Used': exp_date.date(),
        'T (yrs)': T,
        'Market Dirty': P_mkt_dirty,
        'Noncallable Dirty': P_nc_dirty,
        'Forward Price at Expiry': F,
        'Call PV (Black)': opt_pv,
        'Callable Dirty': P_call_model,
        'Model - Market': P_call_model - P_mkt_dirty
    })

out_18 = pd.DataFrame(rows)
display(out_18.style.format({
    'Market Dirty': '${:,.2f}',
    'Noncallable Dirty': '${:,.2f}',
    'Forward Price at Expiry': '${:,.2f}',
    'Call PV (Black)': '${:,.2f}',
    'Callable Dirty': '${:,.2f}',
    'Model - Market': '${:,.2f}'
}))


,Bond,Expiry Used,T (yrs),Market Dirty,Noncallable Dirty,Forward Price at Expiry,Call PV (Black),Callable Dirty,Model - Market
0,FHLMC 0.97 01/28/28,2025-07-28,0.452055,$90.19,$91.28,$92.60,$0.00,$91.28,$1.09
1,FHLMC 1 1/4 01/29/30,2025-07-29,0.454795,$85.16,$87.23,$88.34,$0.00,$87.23,$2.07


Since the calls are far out of the money, the callable & noncallable prices coincide and the. bonds display essentially no callable-related convexity. 

### 1.9. ATM with 1-yr expiry

Try this alternate file `2025-02-18` for a recently-issued bond of size $1bn with a one-year expiration.
* Easier to see the negative convexity.
* Large size, recency should be more liquid.


In [ ]:
FILE_BOND = 'callable_bonds_2025-02-18.xlsx'
FILE_CURVE = 'discount_curve_2025-02-18.xlsx'
KEY_CALLABLE = 'FHLMC 4.55 02/11/28'

In [209]:
info_19 = pd.read_excel(FILE_BOND,sheet_name='info').set_index('info')
quotes_19 = pd.read_excel(FILE_BOND,sheet_name='quotes').set_index('quotes')

info_19_core = info_19[[KEY_CALLABLE]]
quotes_19_core = quotes_19[[KEY_CALLABLE]]

discs_19 = pd.read_excel(FILE_CURVE,sheet_name='discount curve').set_index('ttm').sort_index()

if 0.0 not in discs_19.index:
    discs_19.loc[0.0, 'discount'] = 1.0
    if 'spot rate' in discs_19.columns:
        discs_19.loc[0.0, 'spot rate'] = 0.0
discs_19 = discs_19.sort_index()

curve_19 = discs_19.copy()
settle_19 = quotes_19_core.loc['Date Quoted', KEY_CALLABLE]
cpn_rate_19 = float(info_19_core.loc['Cpn Rate', KEY_CALLABLE])
freq_19 = int(info_19_core.loc['Cpn Freq', KEY_CALLABLE])
maturity_19 = info_19_core.loc['Date Matures', KEY_CALLABLE]
call_date_19 = info_19_core.loc['Date Next Call', KEY_CALLABLE]
call_price_19 = float(info_19_core.loc['Strike', KEY_CALLABLE])

accrued_19 = float(quotes_19_core.loc['Accrued Interest', KEY_CALLABLE])
market_clean_19 = float(quotes_19_core.loc['Clean Price', KEY_CALLABLE])
market_dirty_19 = float(quotes_19_core.loc['Dirty Price', KEY_CALLABLE])

print('1.9 Inputs Check: \n')
print(f'Settle (Date Quoted): {settle_19}')
print(f'Coupon Rate: {cpn_rate_19:.4%}')
print(f'Coupon Frequency: {freq_19} per year')
print(f'Maturity Date: {maturity_19.date()}')
print(f'Call Date: {call_date_19.date()}')
print(f'Call Price (Strike): ${call_price_19:.2f} per 100.0')
print(f'Accrued Interest: ${accrued_19:.2f}')
print(f'Market Clean Price: ${market_clean_19:.2f}')
print(f'Market Dirty Price: ${market_dirty_19:.2f}')

1.9 Inputs Check: 

Settle (Date Quoted): 2025-02-13 00:00:00
Coupon Rate: 4.4100%
Coupon Frequency: 2 per year
Maturity Date: 2030-01-28
Call Date: 2028-01-28
Call Price (Strike): $100.00 per 100.0
Accrued Interest: $0.20
Market Clean Price: $99.89
Market Dirty Price: $100.09


For the recently issued Freddie Mac callable with a one-year expiry, the forward price at the call date is close to the call strike, making the embedded option near at-the-money. As a result, the call option has a materially positive value and the callable bond exhibits pronounded negative convexity, with effective uration significantly lower than that of the hypothetical noncallable bond. This effect is much stronger than in the far out-of-the-money examples, illustrating how callable risk becomes economically important when the option is near the money and short-dated.